# **Cálculo do Preço Médio de Ações**

## **Importando Bibliotecas**

In [1]:
import pandas as pd
import numpy as np

## **Importando Datasets**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
empresas = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Portfólio/Auxiliar_IRPF/Dados_CNPJ_Empresas.csv')

In [4]:
display(empresas)

,CÓDIGO,RAZÃO SOCIAL,NOME DE PREGÃO,CNPJ,TIPO
0,ADHM1,ADVANCED DIGITAL HEALTH MEDICINA PREVENTIVA S.A.,ADVANCED-DH,10.345.009/0001-98,AÇÃO
1,ADHM3,ADVANCED DIGITAL HEALTH MEDICINA PREVENTIVA S.A.,ADVANCED-DH,10.345.009/0001-98,AÇÃO
2,TIET11,AES TIETE ENERGIA SA,AES TIETE E,04.128.563/0001-10,AÇÃO
3,TIET3,AES TIETE ENERGIA SA,AES TIETE E,04.128.563/0001-10,AÇÃO
4,TIET4,AES TIETE ENERGIA SA,AES TIETE E,04.128.563/0001-10,AÇÃO
...,...,...,...,...,...
870,XPCM11,XP CORPORATE MACAÉ FDO INV IMOB – FII,FII XP MACAE,16.802.320/0001-03,FII
871,XPML11,XP MALLS FDO INV IMOB FII,FII XP MALLS,28.757.546/0001-00,FII
872,XPPR11,XP PROPERTIES FDO. INV. IMOB.,FII XP PROP,30.654.849/0001-40,FII
873,XPSF11,XP SELECTION FDO DE FUNDOS INV IMOB – FII,FII XP SELEC,30.983.020/0001-90,FII


In [19]:
movimentacao_carteira = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Portfólio/Auxiliar_IRPF/Movimentacoes_Carteira - Leiri.xlsx')
movimentacao_carteira.dropna(inplace=True)

In [15]:
ano_base = '2020'

In [20]:
movimentacao_carteira = movimentacao_carteira[movimentacao_carteira['DATA'] <= pd.to_datetime(ano_base+'-12-31')]
movimentacao_carteira.reset_index(inplace=True, drop=True)

In [21]:
display(movimentacao_carteira)

,CÓDIGO,DATA,VOLUME,OPERAÇÃO,PREÇO,TAXAS
0,VVAR3,2019-08-16,1,Compra,7.60,0.00
1,ITSA4,2019-10-07,3,Compra,12.43,0.01
2,BBDC4,2019-10-08,1,Compra,32.77,0.01
3,ITSA4,2019-10-08,1,Compra,12.28,0.00
4,BBDC4,2020-03-12,1,Compra,21.32,0.01
5,BBDC4,2020-03-12,1,Compra,21.30,0.00
6,ITSA4,2020-03-12,6,Compra,9.15,0.01
7,PETR4,2020-03-12,2,Compra,12.58,0.01
8,WEGE3,2020-03-12,1,Compra,34.49,0.01
9,BBDC4,2020-04-01,1,Compra,19.44,0.01


## **Cálculo do Preço Médio**


In [9]:
def calcula_preco_medio(compras, vendas):
  compras['VALOR PAGO'] = compras['PREÇO'] * compras['VOLUME'] + compras['TAXAS']
  vendas['VALOR PAGO'] = (vendas['PREÇO'] * vendas['VOLUME'])* -1 + vendas['TAXAS']
  volume = compras['VOLUME'].sum() - vendas['VOLUME'].sum()
  preco_medio = np.round(((compras['VALOR PAGO'].sum() + vendas['VALOR PAGO'].sum()) / volume), decimals=2)
  return preco_medio, volume

In [10]:
carteira = []
for t in np.sort(movimentacao_carteira['CÓDIGO'].unique()):
  acao = movimentacao_carteira[movimentacao_carteira['CÓDIGO']==t]
  preco_medio, volume = calcula_preco_medio(acao[acao['OPERAÇÃO']=='Compra'], acao[acao['OPERAÇÃO']=='Venda'])
  carteira.append([t, preco_medio, volume, np.round(preco_medio * volume, decimals=2)])

In [11]:
linhas = []
for i in range(len(carteira)):
  cnpj = empresas[empresas['CÓDIGO']==carteira[i][0]]
  if cnpj['TIPO'].to_string(index=False) == ' AÇÃO':
    if carteira[i][2] > 1:
      linhas.append(str(int(carteira[i][2])) + ' AÇÕES DA EMPRESA' +  cnpj['RAZÃO SOCIAL'].to_string(index=False) + ', CNPJ' + 
            cnpj['CNPJ'].to_string(index=False) + ', CÓDIGO DE NEGOCIAÇÃO' + cnpj['CÓDIGO'].to_string(index=False) +
            ' NEGOCIADAS NA B3 E ADQUIRIDAS PELO VALOR TOTAL DE R$ ' + str(carteira[i][3]) + ' E A UM CUSTO MÉDIO DE R$ ' + str(carteira[i][1]) + '.\n\n')
    else:
      linhas.append(str(int(carteira[i][2])) + ' AÇÃO DA EMPRESA' +  cnpj['RAZÃO SOCIAL'].to_string(index=False) + ', CNPJ' + 
            cnpj['CNPJ'].to_string(index=False) + ', CÓDIGO DE NEGOCIAÇÃO' + cnpj['CÓDIGO'].to_string(index=False) +
            ' NEGOCIADA NA B3 E ADQUIRIDA PELO VALOR TOTAL DE R$ ' + str(carteira[i][3]) + ' E A UM CUSTO MÉDIO DE R$ ' + str(carteira[i][1]) + '.\n\n')
  else:
    if carteira[i][2] > 1:
      linhas.append(str(int(carteira[i][2])) + ' COTAS DO FII' +  cnpj['RAZÃO SOCIAL'].to_string(index=False) + ', CNPJ' + 
            cnpj['CNPJ'].to_string(index=False) + ', CÓDIGO DE NEGOCIAÇÃO' + cnpj['CÓDIGO'].to_string(index=False) +
            ' NEGOCIADAS NA B3 E ADQUIRIDAS PELO VALOR TOTAL DE R$ ' + str(carteira[i][3]) + ' E A UM CUSTO MÉDIO DE R$ ' + str(carteira[i][1]) + '.\n\n')
    else:
      linhas.append(str(int(carteira[i][2])) + ' COTA DO FII' +  cnpj['RAZÃO SOCIAL'].to_string(index=False) + ', CNPJ' + 
            cnpj['CNPJ'].to_string(index=False) + ', CÓDIGO DE NEGOCIAÇÃO' + cnpj['CÓDIGO'].to_string(index=False) +
            ' NEGOCIADAS NA B3 E ADQUIRIDAS PELO VALOR TOTAL DE R$ ' + str(carteira[i][3]) + ' E A UM CUSTO MÉDIO DE R$ ' + str(carteira[i][1]) + '.\n\n')

In [12]:
arquivo = open('/content/drive/MyDrive/Colab Notebooks/Portfólio/Auxiliar_IRPF/Preco_Medio_IR - Leiri.txt', 'a')
arquivo.writelines(linhas)
arquivo.close()

In [22]:
np.sort(movimentacao_carteira['CÓDIGO'].unique())

array(['ALZR11', 'BBDC4', 'BTLG11', 'CTSA4', 'ITSA4', 'ITUB4', 'PETR4',
       'VISC11', 'VVAR3', 'WEGE3', 'XPML11'], dtype=object)